In [0]:
#Read Manual here: \\corpaa.aa.com\campusshared\HDQ\HDQ_REVMGMT_Share\RMDEPT\Revenue Analysis\Atlantic RA\Adhoc Projects\2024.01 RA Trip Intent Model\Manual-read_me

#Import all required packages

install the resampling package for ML

In [0]:
pip install imblearn

In [0]:
pip install -U scikit-learn 

import all needed packages. Use your own user id for mosaic for adhoc code tuning

In [0]:
#put all required modules here
from murph import mosaic, rm_fileshare
import numpy as np 
import pandas as pd 
import datetime as dt
user = "866810"
mos = mosaic(user = user, tmode='TERA')
group_name = 'group110'
fileshare = rm_fileshare(group_name)
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support
# the provided feature data time range is 10/01/2022 to 09/30/2023. Be consistant with this for all data pulling

# Data Reading and Processing

In [0]:
features_path = fileshare.path(directory_name="ra/projects/RA_trip_Intent_Model", file_name="features_20221001_to_20230930.csv")
or_features = pd.read_csv(features_path,on_bad_lines='skip')
or_features.columns = or_features.columns.str.lower()
or_features.head(10)

In [0]:
#get TA data and NA pos
or_features_eu_only = or_features.loc[(or_features.rm_region_eu_ind == 1)&(or_features.tkt_cntry_issue_cd.isin(['US','CA','PR','VI']))].copy()
or_features_eu_only.shape

In [0]:
#save  to csv
fileshare.to_csv(or_features_eu_only, "ra/projects/RA_Trip_Intent_Model/OR_features_eu_only.csv", index=False)

In [0]:
or_features_eu_only.shape

## Import EU/TA feature data from Original OR features 
####This data only include features with rm_eu_indicator = 1. EU and GB/UK are included.

In [0]:
#read EU data and lower strings of all column names
features_path = fileshare.path(directory_name="ra/projects/RA_Trip_Intent_Model", file_name="OR_features_eu_only.csv")
or_features = pd.read_csv(features_path)
or_features.columns = or_features.columns.str.lower()
or_features.head(10)

In [0]:
#first selection of sebset of features. The original data contains 300+ features
or_features_short = or_features[[

'pnr_loctr_id',
'pnr_create_dt',
'pnr_od',
'pnr_ndod',
'pnr_dep_dt',
'pnr_dep_dow',
'pnr_return_dt',
'pnr_return_dow',
'trip_duration',
'round_trip_type',
'trip_type',
'ap',
'pnr_origin_airprt_iata_cd',
'pnr_origin_metro_city_cd',
'pnr_origin_cntry_cd',
'pnr_destntn_airprt_iata_cd',
'pnr_destntn_metro_city_cd',
'pnr_destntn_cntry_cd',
'connecting_ind',
'not_using_closest_airprt',
'dep_return_airprt_different_ind',
'pnr_premium_cabin_ind', #whether any passenger in pnr was booked in WCF cabins
'pnr_max_cabin_cd',
'pnr_max_lylty_level_num',
'tkt_cntry_issue_cd', #may need to filter on this to be 'US' for the following analysis
 'group_size',
 'prime_chanl_type_cd',

 #model labels
 'trip_subpurpose',
 'or_trip_purpose',
'css_trip_purpose',

                          ]]

In [0]:
or_features_short.shape

In [0]:
#create a few variable based on existing or features
or_features_short.rename(columns = {'trip_duration':'tkt_min_stay'}, inplace= True)
or_features_short.pnr_create_dt = or_features_short.pnr_create_dt.astype("datetime64[ns]")
or_features_short['pnr_return_dt'] = pd.to_datetime(or_features_short['pnr_return_dt'], format='%Y/%m/%d')
or_features_short['pnr_dep_dt'] = pd.to_datetime(or_features_short['pnr_dep_dt'], format='%Y/%m/%d')
or_features_short['pnr_create_dt'] = pd.to_datetime(or_features_short['pnr_create_dt'], format='%Y/%m/%d')
# dayofweek will return day of week with Monday = 0 and Sunday = 7. Adding 1 to make Monday = 1 and Sunday = 7
or_features_short['dep_dow_modified'] = or_features_short['pnr_dep_dt'].dt.dayofweek + 1
or_features_short['return_dow_modified'] = or_features_short['pnr_return_dt'].dt.dayofweek + 1
or_features_short['dep_dow_modified'] = or_features_short[ 'dep_dow_modified'].astype(float)
or_features_short['return_dow_modified'] = or_features_short[ 'return_dow_modified'].astype(float)

#create pax count indicator per pnr
or_features_short['pnr_single_pax_count_indicator'] = or_features_short.group_size.apply(lambda x: 1 if x == 1 else 0)

#create country as non-US country code
def country(row):
    if row['pnr_origin_cntry_cd'] in (['US','MX','CA']):
      return row['pnr_destntn_cntry_cd']
    else:
      return row['pnr_origin_cntry_cd']
or_features_short['country'] = or_features_short.apply(country, axis = 1)

#create mkt group as UK, core EU and others
def mkt_group(row):
  if row['country'] == 'GB':
    return 'UK'
  elif row['country'] in (['DE','FR','GR','IT','NL','IE','CH','CZ','HR','HU','IS','PT','ES','FI']):
    return 'Core EU'
  else:
    return 'Others'
or_features_short['mkt_group'] = or_features_short.apply(mkt_group, axis = 1)

#creating TKT MIN STAY

def func_min_stay(row):
  if row['tkt_min_stay'] >= 28:
    return '28+ NT'
  if row['tkt_min_stay'] >= 21 and row['tkt_min_stay']<28:
    return '21-27 NT'
  elif row['tkt_min_stay']>= 14 and row['tkt_min_stay'] <= 20:
    return '14-20 NT'
  elif row['tkt_min_stay']>= 10 and row['tkt_min_stay']<=13:
    return '10-13 NT'
  elif row['tkt_min_stay']>=7 and row['tkt_min_stay'] <= 9:
    return '7-9 NT'
  elif row['tkt_min_stay']>=1 and row['tkt_min_stay'] <=6:
    if row['dep_dow_modified'] ==1 and row['tkt_min_stay'] >5:
      return 'SUN'
    elif row['dep_dow_modified'] ==2 and row['tkt_min_stay'] >4:
      return 'SUN'
    elif row['dep_dow_modified'] ==3 and row['tkt_min_stay'] >3:
      return 'SUN'
    elif row['dep_dow_modified'] ==4 and row['tkt_min_stay'] >2:
      return 'SUN'
    elif row['dep_dow_modified'] ==5 and row['tkt_min_stay'] >1:
      return 'SUN'
    elif row['dep_dow_modified'] ==6 and row['tkt_min_stay'] >0:
      return 'SUN'
    else:
      return 'NO SUN'
  elif row['tkt_min_stay'] ==0:
    return 'Same Day Roundtrip'
  elif row['round_trip_type'] == 'OUTBOUND ONE-WAY':
    return 'One Way'
  elif row['round_trip_type'] == 'ONE-WAY':
    return 'One Way'
  elif  row['round_trip_type'] == 'INBOUND ONE-WAY':
    return 'One Way'
  else:
    return 'Error'
or_features_short['tkt_min_stay_category'] = or_features_short.apply(func_min_stay, axis = 1)


#creating 3 category trip intent label
def func_trip_intent_3_cat_label(row):
  if row['trip_subpurpose'] == 'BUSINESS_ONLY':
    return 'Business Only'
  elif row['trip_subpurpose'] == 'VACATION_ONLY':
    return 'Vacation Only'
  else:
    return 'Blended'
or_features_short['trip_subpurpose_3c'] = or_features_short.apply(func_trip_intent_3_cat_label, axis = 1)

#creating 2 category trip intent label (based on subpurpose, not copying css trip purpose)
def func_trip_intent_2_cat_label(row):
  if row['trip_subpurpose'] == 'BUSINESS_ONLY':
    return 'Business'
  elif row['trip_subpurpose'] == 'BLEISURE':
    return 'Business'
  elif row['trip_subpurpose'] == 'VACATION_ONLY':
    return 'Leisure'
  elif row['trip_subpurpose'] == 'PERSONAL_OTHER':
    return 'Leisure'
  elif row['trip_subpurpose'] == 'VFR':
    return 'Leisure'
  else:
    return 'Error'
or_features_short['trip_subpurpose_2c'] = or_features_short.apply(func_trip_intent_2_cat_label, axis = 1)

#creating AP category
def func_ap_category(row):
  if row['ap'] < 7:
    return '0-6 AP'
  elif row['ap'] < 14:
    return '7-13 AP'
  elif row['ap'] < 21:
    return '14-20 AP'
  elif row['ap'] < 28:
    return '21-27 AP'
  elif row['ap'] < 50:
    return '28-49 AP'
  elif row['ap'] < 90:
    return '50-89 AP'
  elif row['ap'] < 120:
    return '90-119 AP'
  elif row['ap'] < 150:
    return '120-149 AP'
  elif row['ap'] < 366:
    return '150-365 AP'
  elif row['ap'] >= 366:
    return '365+ AP'
  else:
    return 'Error'
or_features_short['tkt_ap_category'] = or_features_short.apply(func_ap_category, axis = 1)


#adding flight quarter/year and booking quarter/year. This may help to represent seasonality
or_features_short['flt_dep_quarter'] = or_features_short.pnr_dep_dt.dt.quarter
or_features_short['pnr_booking_quarter'] = or_features_short.pnr_create_dt.dt.quarter
#or_features_short['flt_dep_quarter'] = or_features_short.PeriodIndex(or_features_short.pnr_dep_dt, freq='Q')
#or_features_short['pnr_booking_quarter'] = or_features_short.PeriodIndex(or_features_short.pnr_create_dt, freq='Q')



In [0]:
or_features_short.tkt_min_stay_category.value_counts()


In [0]:
#save feature_short to csv
fileshare.to_csv(or_features_short, "ra/projects/RA_Trip_Intent_Model/ta_or_features_short.csv", index=False)

In [0]:
or_features_short.head(5)

In [0]:
#creating some extra variables which are controlable by pricing. e.g.FBC
#YM_info = mos.read_sql_to_pandas(
YM_info = mos.read_sql(
"""

  SELECT
TRIM(PNR_LOCTR_ID) AS PNR_LOCTR_ID,
PNR_CREATE_DT,
TRIM(OD_AIRPRT_PAIR_CD) AS OD_AIRPRT_PAIR_CD,
TRNSATL_FARE_BASE_CD,
SUBSTR(TRNSATL_FARE_BASE_CD,1,1) AS FBC1,
SUBSTR(TRNSATL_FARE_BASE_CD,2,1) AS FBC2,
SUBSTR(TRNSATL_FARE_BASE_CD,7,1) AS FBC7,
SUBSTR(TRNSATL_FARE_BASE_CD,8,1) AS FBC8,
CASE WHEN SUBSTRING(TRIM(TRNSATL_FARE_BASE_CD),7,2) IN (
  'Z1', 'Z3', 'Z6','BV','B4','B5') THEN 1 ELSE 0 END AS BASIC_ECONOMY_IND,
COALESCE(YM_HUB_NM, 'OTHER') AS YM_HUB_NM,
COALESCE(YM_SPOKE_NM, 'OTHER') AS YM_SPOKE_NM,
--COUNT(*) AS TKT_CNT
SUM(OD_PAX_QTY) AS ONE_WAY_PAX_QTY,
SUM(OD_NET_YQ_REV_USD_AMT) AS ONE_WAY_REV

FROM PROD_JBA_VW.JBA_FLOWN_OD_DETAIL A

LEFT JOIN (
SELECT
DISTINCT
LEG_DIRECTN_AIRPRT_IATA_CD,
YM_HUB_NM,
YM_SPOKE_NM
FROM PROD_RM_BUSINES_VW.MIRS_TRUE_OD_FLOW_DETL
WHERE TICKET_RPT_DT BETWEEN '2022/10/01' AND '2023/09/30'
AND OD_TICKET_RPT_DT BETWEEN '2022/10/01' AND '2023/09/30'
AND YM_HUB_NM IS NOT NULL
AND YM_SPOKE_NM IS NOT NULL
) SPOKE
ON A.OD_AIRPRT_PAIR_CD = SPOKE.LEG_DIRECTN_AIRPRT_IATA_CD
WHERE 
1=1
AND POA_PNR_CNTRY_CD IN ('US','CA','PR','VI','MX')
AND TICKET_POS_REGION_CD = 'NA'
AND TRNSATL_MKT_AIRLN_IATA_CD= 'AA'
AND TRNSATL_OPERAT_AIRLN_IATA_CD IN ('AA', 'BA','IB','AY')
AND TRNSATL_NET_REV_USD_AMT > 0
AND LENGTH(TRNSATL_FARE_BASE_CD) = 8
AND OD_DEP_DT BETWEEN '2022-10-01' AND '2023-09-30'
AND PNR_CREATE_DT >= '2021-10-01'
GROUP BY 1,2,3,4,5,6,7,8,9,10,11
"""
)
YM_info.createOrReplaceTempView("YM_info")

In [0]:
rtpq = spark.sql(
"""
WITH raw AS (
SELECT
--SNAPSHOT_DATE,
PNR_CREATION_DATE AS PNR_CREATE_DT,
TRIM(PNR) AS PNR_LOCTR_ID,
--OD_ORIG,
--OD_DEST,
TRIM(OD_ORIG)||TRIM(OD_DEST) AS OD,
--PER_PAX_FARE,
--FARE_WITH_SURCHARGE,
--TOTAL_TAX,
PER_PAX_SALES_TOTAL,
--PURCHASED_FARE_BRAND_CD,
PRODUCT_BRAND_CD,
--PRIME_CHANL_TYPE_CD,
TERTY_CHANL_TYPE_CD
FROM ym_hist.rtpq2_od_hist
WHERE 
1=1
AND SNAPSHOT_DATE = PNR_CREATION_DATE
AND PNR_CREATION_DATE >= '2021-10-01'
AND FLIGHT_DEP_DT BETWEEN '2022-10-01' AND '2023-09-30'
),
middle AS (
  SELECT 
PNR_LOCTR_ID,
PNR_CREATE_DT,
OD,
PER_PAX_SALES_TOTAL,
PRODUCT_BRAND_CD,
TERTY_CHANL_TYPE_CD,
ROW_NUMBER() OVER (PARTITION BY PNR_LOCTR_ID,PNR_CREATE_DT, OD ORDER BY PER_PAX_SALES_TOTAL DESC) AS PNR_ROW
FROM raw
)
SELECT
PNR_LOCTR_ID,
PNR_CREATE_DT,
OD,
PER_PAX_SALES_TOTAL,
PRODUCT_BRAND_CD,
TERTY_CHANL_TYPE_CD
FROM middle
WHERE PNR_ROW = 1
"""
)
rtpq.createOrReplaceTempView("rtpq")

In [0]:
product = spark.sql(
  """
SELECT
A.PNR_LOCTR_ID,
A.PNR_CREATE_DT,
A.OD AS PNR_OD,
A.PRODUCT_BRAND_CD,
A.TERTY_CHANL_TYPE_CD,
B.BASIC_ECONOMY_IND,
B.YM_HUB_NM,
B.YM_SPOKE_NM,
B.ONE_WAY_PAX_QTY,
B.ONE_WAY_REV
FROM rtpq A
JOIN YM_info B
ON  A.PNR_LOCTR_ID = B.PNR_LOCTR_ID AND A.PNR_CREATE_DT = B.PNR_CREATE_DT

  """
)
product.createOrReplaceTempView("product")

In [0]:
product=product.toPandas()

In [0]:
type(product)
product.columns = product.columns.str.lower()
#product.rename(columns = {'od':'pnr_od'}, inplace = True)
product.pnr_create_dt = product.pnr_create_dt.astype("datetime64[ns]")

In [0]:
features_short_joint = or_features_short.merge(product, how = 'left', on = ['pnr_loctr_id', 'pnr_create_dt','pnr_od'])

In [0]:
features_short_joint.shape

In [0]:
#FILL na and remove duplicated rows
na_values = {"product_brand_cd": 'Other', "terty_chanl_type_cd": 'Other', "ym_hub_nm": 'Other', "ym_spoke_nm": 'Other','basic_economy_ind':0}
features_short_joint = features_short_joint.fillna(value=na_values)
features_short_joint.drop_duplicates(inplace=True)
features_short_joint.shape


In [0]:
features_short_joint.pnr_max_cabin_cd.value_counts()
features_short_joint.basic_economy_ind.value_counts()
#features_short_joint.product_brand_cd.value_counts()

In [0]:
features_short_joint.head()

In [0]:
#save features_short_joint to csv
fileshare.to_csv(features_short_joint, "ra/projects/RA_Trip_Intent_Model/ta_features_short_joint.csv", index=False)

#Build 3 Category Model

In [0]:
#read features data and show all column names in features_short_joint
model_raw_data_path = rm_fileshare().path(directory_name="ra/users/866810/trip_intent_model", file_name="features_short_joint.csv")
model_raw_data = pd.read_csv(model_raw_data_path)

na_values = {"product_brand_cd": 'Other', "terty_chanl_type_cd": 'Other', "ym_hub_nm": 'Other', "ym_spoke_nm": 'Other','basic_economy_ind':0}
model_raw_data = model_raw_data.fillna(value=na_values)
model_raw_data.drop_duplicates(inplace=True)
print(model_raw_data.shape)
print(model_raw_data.columns)

In [0]:
model_data = model_raw_data[[
  # feature data, numeric, ordinal categorical or indicator
'connecting_ind',
'not_using_closest_airprt',
'dep_return_airprt_different_ind',
    #'pnr_premium_cabin_ind',
    #'pnr_max_lylty_level_num',
'pnr_single_pax_count_indicator',
'ap',
'tkt_min_stay',
'basic_economy_ind',
  # feature data, categorical data, need to be converted
'round_trip_type',
'trip_type',
'prime_chanl_type_cd',
'dep_dow_modified',
'return_dow_modified',
'mkt_group',
'tkt_min_stay_category',
'tkt_ap_category',
#'product_brand_cd',
'ym_hub_nm',
'ym_spoke_nm',
'flt_dep_quarter',
    #'terty_chanl_type_cd'
'pnr_max_cabin_cd',
  # label data
'trip_subpurpose',
'css_trip_purpose',
'trip_subpurpose_3c',
'trip_subpurpose_2c'


]]

In [0]:
#generate trip intent predicted label to compare against with CSS GT label and calculate the new OR trip intent model accuracy performance
trip_intent = mos.read_sql(
  """
  SELECT
  top 10000000
  trim(PNR_LOCTR_ID) as PNR_LOCTR_ID,
  TRIM(PNR_CREATE_DT) AS PNR_CREATE_DT,
  PNR_CREATE_TM,
  CASE Greatest(Coalesce(FLOWN_BUSINES_ONLY_PROB_PCT, BKG_BUSINES_ONLY_PROB_PCT),
    Coalesce(FLOWN_BLEISURE_PROB_PCT, BKG_BLEISURE_PROB_PCT),
    Coalesce(FLOWN_VFR_PROB_PCT, BKG_VFR_PROB_PCT),
    Coalesce(FLOWN_VACTN_ONLY_PROB_PCT, BKG_VACTN_ONLY_PROB_PCT),
    Coalesce(FLOWN_PERSNL_OTHR_PROB_PCT, BKG_PERSNL_OTHR_PROB_PCT))
  WHEN Coalesce(FLOWN_BUSINES_ONLY_PROB_PCT, BKG_BUSINES_ONLY_PROB_PCT) THEN 'BUSINESS_ONLY'
  WHEN Coalesce(FLOWN_BLEISURE_PROB_PCT, BKG_BLEISURE_PROB_PCT) THEN 'BLEISURE'
  WHEN Coalesce(FLOWN_VFR_PROB_PCT, BKG_VFR_PROB_PCT) THEN 'VFR'
  WHEN Coalesce(FLOWN_VACTN_ONLY_PROB_PCT, BKG_VACTN_ONLY_PROB_PCT) THEN 'VACATION_ONLY'
  WHEN Coalesce(FLOWN_PERSNL_OTHR_PROB_PCT, BKG_PERSNL_OTHR_PROB_PCT) THEN 'PERSONAL_OTHER'
  END AS TRIP_INTENT_LABEL,
  Coalesce(FLOWN_BUSINES_ONLY_PROB_PCT, BKG_BUSINES_ONLY_PROB_PCT) AS BUSINES_ONLY_PROB_PCT,
  Coalesce(FLOWN_BLEISURE_PROB_PCT, BKG_BLEISURE_PROB_PCT) AS BLEISURE_PROB_PCT,
  Coalesce(FLOWN_VFR_PROB_PCT, BKG_VFR_PROB_PCT) AS VFR_PROB_PCT,
  Coalesce(FLOWN_VACTN_ONLY_PROB_PCT, BKG_VACTN_ONLY_PROB_PCT) AS VACTN_ONLY_PROB_PCT,
  Coalesce(FLOWN_PERSNL_OTHR_PROB_PCT, BKG_PERSNL_OTHR_PROB_PCT) AS PERSNL_OTHR_PROB_PCT
  FROM PROD_PNR_BOOKING_VWS.PNR_TRIP_INTENT
  WHERE PRELIM_SCORE_IND = 'N'
  AND PNR_CREATE_DT between '2021/02/23' and '2023/08/31'
  """
)
trip_intent.createOrReplaceTempView("trip_intent")

In [0]:
trip_intent = trip_intent.toPandas()

In [0]:
trip_intent.columns = trip_intent.columns.str.lower()

##estimate OR model accuracy

In [0]:
test_accuracy = model_raw_data.merge(trip_intent, how = 'left', on = ['pnr_loctr_id', 'pnr_create_dt'])

In [0]:
test_accuracy=test_accuracy[test_accuracy['trip_subpurpose'].isin(['VFR','BUSINESS_ONLY','VACATION_ONLY'])].copy() 

In [0]:
#estimate the prediction accuracy of current OR 5-category trip intent model ~82.54%
#100* test_accuracy.shape[1] / test_accuracy[test_accuracy.trip_subpurpose == test_accuracy.trip_intent_label].shape[1]
100*test_accuracy[test_accuracy.trip_subpurpose == test_accuracy.trip_intent_label].shape[0]/test_accuracy[~pd.isna(test_accuracy.trip_intent_label)].shape[0]

In [0]:
100*test_accuracy[(test_accuracy.trip_subpurpose == test_accuracy.trip_intent_label) & (test_accuracy.trip_intent_label =='VFR') &(test_accuracy.trip_subpurpose =='VFR')].shape[0]/test_accuracy[(~pd.isna(test_accuracy.trip_intent_label)) & (test_accuracy.trip_intent_label =='VFR')].shape[0]

## build a single tree model

In [0]:
#convert category variables into dummies for model training


model_data_one_hot = pd.get_dummies(model_data, columns = [
'round_trip_type',
'trip_type',
'prime_chanl_type_cd',
'dep_dow_modified',
'return_dow_modified',
'mkt_group',
'tkt_min_stay_category',
'tkt_ap_category',
'pnr_max_cabin_cd',
'ym_hub_nm',
'ym_spoke_nm',
'flt_dep_quarter'
  ])
#print(model_data_one_hot)
print(model_data_one_hot.columns.tolist())
model_data_one_hot = model_data_one_hot.dropna()
model_data_one_hot.shape

In [0]:
model_data_one_hot=model_data_one_hot[model_data_one_hot['trip_subpurpose'].isin(['VFR','BUSINESS_ONLY','VACATION_ONLY'])].copy()

In [0]:

le = preprocessing.LabelEncoder()


model_data_one_hot['trip_subpurpose'] = le.fit_transform(model_data_one_hot.trip_subpurpose)

print(le.classes_)



#model_data_one_hot['trip_subpurpose'] = preprocessing.LabelEncoder().fit_transform(model_data_one_hot['trip_subpurpose'])

In [0]:
model_data_one_hot.trip_subpurpose.value_counts()

In [0]:
#split training and testing data
X = model_data_one_hot.drop(columns = ['trip_subpurpose_3c','css_trip_purpose','trip_subpurpose_2c','trip_subpurpose'])
y = model_data_one_hot['trip_subpurpose']
# dividing X, y into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [0]:

ros = RandomOverSampler(random_state=0)
#X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)

In [0]:
#X_resampled.shape
y_resampled.value_counts()
#X_test.shape

In [0]:
single_tree_model4 = DecisionTreeClassifier(max_depth = 20).fit(X_resampled, y_resampled)
single_tree_predictions4 = single_tree_model4.predict(X_test)
single_tree_pred_prob4 = single_tree_model4.predict_proba(X_test)
# creating a confusion matrix
cm4 = confusion_matrix(y_test, single_tree_predictions4)

In [0]:
#plot the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm4,display_labels=single_tree_model4.classes_)
disp.plot()

In [0]:
#accuracy calculation
feature_name4 = X.columns
class_name4 =single_tree_model4.classes_
#accuracy_score(y_test, single_tree_predictions)
predict_prob4 = (np.sum(single_tree_pred_prob4, axis= 0)/len(y_test))
predict_prob4 = pd.DataFrame((predict_prob4), index = single_tree_model4.classes_)
true_prob4 = pd.DataFrame(y_test.value_counts()/len(y_test))
result4 = pd.concat([predict_prob4, true_prob4], axis=1).reindex(predict_prob4.index)
result4.columns = ['Predicted Prob', 'True Prob']
result4['Absolute Bias'] =abs( (result4['Predicted Prob'] - result4['True Prob'] )/result4['True Prob'])
single_tree_fitting_predictions4 = single_tree_model4.predict(X_resampled)
print('label based TRAINING accuracy',accuracy_score(y_resampled, single_tree_fitting_predictions4) )
print('label based PREDICTING accuracy is', accuracy_score(y_test, single_tree_predictions4))
print('Aggregated Probability based PREDICTING accuracy is\n', result4)

In [0]:
precision, recall, fscore, support = precision_recall_fscore_support(y_test, single_tree_predictions4)
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))

In [0]:
#plot the tree
feature_name = X.columns
class_name =single_tree_model4.classes_
fig = plt.figure(figsize=(35,35))
_ = tree.plot_tree(single_tree_model4, 
                   feature_names=feature_name,  
                   class_names=class_name,
                   filled=True,
                   fontsize = 15)

In [0]:
#plot importance ranks
feature_importances = pd.DataFrame(single_tree_model4.feature_importances_, index=X.columns, columns=["Importance"])
feature_importances.sort_values(by='Importance', ascending=False, inplace=True)
feature_importances[:9].plot(kind='bar', figsize=(8,6))

##build random forest

In [0]:
# build RF
#build a forest of 100 trees
forest2 = RandomForestClassifier(n_estimators=100,random_state=0, max_depth=20)
forest_model2 = forest2.fit(X_resampled, y_resampled)
RF_feature_name = X.columns
RF_class_name =forest2.classes_


In [0]:
#generate predictions
RF_predictions2 = forest_model2.predict(X_test)
RF_pred_prob2 = forest_model2.predict_proba(X_test)
#calculate performance
RF_predict_prob2 = (np.sum(RF_pred_prob2, axis= 0)/len(y_test))
RF_predict_prob_df2 = pd.DataFrame((RF_predict_prob2), index = RF_class_name)
RF_true_prob_df2 = pd.DataFrame(y_test.value_counts()/len(y_test))
RF_result2 = pd.concat([RF_predict_prob_df2, RF_true_prob_df2], axis=1).reindex(RF_predict_prob_df2.index)
RF_result2.columns = ['Predicted Prob', 'True Prob']
RF_result2['Absolute Bias'] =abs( (RF_result2['Predicted Prob'] - RF_result2['True Prob'] )/RF_result2['True Prob'])
RF_fitting_predictions2 = forest_model2.predict(X_resampled)
print('Label based TRAINING accuracy',accuracy_score(y_resampled, RF_fitting_predictions2) )
print('Label based PREDICTING accuracy is', accuracy_score(y_test, RF_predictions2))
print('Aggregated Probability based PREDICTING accuracy is\n', RF_result2)

In [0]:
precision, recall, fscore, support = precision_recall_fscore_support(y_test, RF_predictions2)
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))

In [0]:
cm5 = confusion_matrix(y_test, RF_predictions2)
disp = ConfusionMatrixDisplay(confusion_matrix=cm5,display_labels=forest_model2.classes_)
disp.plot()

In [0]:
#generate feature importance from RF
feature_importances = pd.DataFrame(forest_model2.feature_importances_, index=X.columns, columns=["Importance"])
feature_importances.sort_values(by='Importance', ascending=False, inplace=True)
feature_importances[:19].plot(kind='bar', figsize=(8,6))

#Modeling with 5/7 features and Forecasting AA, AJB PNRs 

##RF with only top 5 features -AP, Min Stay, PNR PAX Cnt, Cabin, Channel. 

####Seasonality and Entity/MKT Group do not contribute hugely to accuracy improvement (~1%ish). Still keep numeric AP and Min Stay for future investigation of optimal cutoff points to help new pricing structure.

In [0]:
#Previous model shows the prediction results are not very accurate for OW PNRs, Non Sun and Sun PNRs. After building models with only top features, the improvement on these segments are minimum. The trick here is to use some high confidence prediction data on these segments from OR predictions as extra training data for our RA Model to force to learn more about these segments. The results show great improvements on accuracy and probability bias.

In [0]:
#Need to run Cmd 1~6 to import necessary packages before running this. Data Reading and 3 Category Model building can be skipped.

In [0]:
#read features data and show all column names in features_short_joint
model_raw_data_path = fileshare.path(directory_name="ra/projects/RA_Trip_Intent_Model", file_name="ta_features_short_joint.csv")
model_raw_data = pd.read_csv(model_raw_data_path)

na_values = {"product_brand_cd": 'Other', "terty_chanl_type_cd": 'Other', "ym_hub_nm": 'Other', "ym_spoke_nm": 'Other','basic_economy_ind':0}
model_raw_data = model_raw_data.fillna(value=na_values)
model_raw_data.drop_duplicates(inplace=True)
print(model_raw_data.shape)
print(model_raw_data.columns)

In [0]:
model_raw_data.mkt_group.value_counts()

In [0]:
model_raw_data.flt_dep_quarter.value_counts()

In [0]:
model_data = model_raw_data[[
 'pnr_loctr_id', 'pnr_create_dt', 
'pnr_single_pax_count_indicator',
'ap',
'tkt_min_stay',
'prime_chanl_type_cd',
'tkt_min_stay_category',
'tkt_ap_category',
'pnr_max_cabin_cd',
#'flt_dep_quarter',
#'mkt_group',
  # label data
'trip_subpurpose',

'group_size'



]]

In [0]:
#a_aa = fileshare.path(directory_name="ra/users/866810/trip_intent_model", file_name="ta_aa_pnr_v3.txt")
ta_aa = fileshare.path(directory_name="ra/projects/RA_Trip_Intent_Model", file_name="ta_aa_pnr_v4_add_mkt.txt")
ta_aa_data = pd.read_csv(ta_aa, sep='	')
ta_aa_data.columns = ta_aa_data.columns.str.lower()
ta_aa_data = ta_aa_data.dropna()

In [0]:
ta_aa_data.head(10)

In [0]:
ta_aa_data_train = ta_aa_data.loc[(ta_aa_data.tkt_min_stay_category =='One Way') & ((ta_aa_data.or_vactn_only_prob_pct > 0.9) | (ta_aa_data.or_busines_only_prob_pct > 0.9)|(ta_aa_data.or_vfr_prob_pct > 0.9))]

In [0]:
ta_aa_data_train_test=ta_aa_data_train[[
 'pnr_loctr_id', 'pnr_create_dt', 
'pnr_single_pax_count_indicator',
'ap',
'tkt_min_stay',
'prime_chanl_type_cd',
'tkt_min_stay_category',
'tkt_ap_category',
'pnr_max_cabin_cd',
'or_label',
'one_way_pax_qty',
#'flt_dep_quarter',
#'mkt_group'
]]

In [0]:
ta_aa_data_train=ta_aa_data_train_test.rename(columns = {"or_label":"trip_subpurpose","one_way_pax_qty":"group_size"}).copy()

In [0]:
model_data=model_data[
  (model_data['trip_subpurpose'].isin(['VFR','BUSINESS_ONLY','VACATION_ONLY'])) 
                  #& (model_data.pnr_single_pax_count_indicator ==1)
                    #  & (model_data['flt_dep_quarter'].isin([2,3]))
                    # & (model_data.mkt_group == 'Core EU')
                      ].copy()

In [0]:
model_data.loc[model_data.tkt_min_stay_category =="One Way","tkt_min_stay"]=0

In [0]:
model_data

In [0]:
ta_aa_data_train

In [0]:
ta_aa_data_train.trip_subpurpose.value_counts()

In [0]:
#turn this line off if do not want to use OR  data
model_data=model_data.append(ta_aa_data_train, ignore_index=True)

In [0]:
model_data=model_data[
  (model_data['trip_subpurpose'].isin(['VFR','BUSINESS_ONLY','VACATION_ONLY'])) 
                  & (model_data.prime_chanl_type_cd.isin(['BUSINESS','DIRECT','LEISURE','OTA','TMC']))
                  #& (model_data.pnr_single_pax_count_indicator ==1)
                    #  & (model_data['flt_dep_quarter'].isin([2,3]))
                    # & (model_data.mkt_group == 'Core EU')
                      ].copy()

In [0]:
#convert category variables into dummies for model training


model_data_one_hot = pd.get_dummies(model_data, columns = [

'prime_chanl_type_cd',
'tkt_min_stay_category',
'tkt_ap_category',
'pnr_max_cabin_cd',
#'flt_dep_quarter',
#'mkt_group'

  ])
#print(model_data_one_hot)
print(model_data_one_hot.columns.tolist())
model_data_one_hot = model_data_one_hot.dropna()
model_data_one_hot.shape

In [0]:
model_data_one_hot=model_data_one_hot[
  (model_data_one_hot['trip_subpurpose'].isin(['VFR','BUSINESS_ONLY','VACATION_ONLY'])) 
                  
                    #  & (model_data['flt_dep_quarter'].isin([2,3]))
                    # & (model_data.mkt_group == 'Core EU')
                      ].copy()

In [0]:
#split training and testing data
X = model_data_one_hot.drop(columns = ['trip_subpurpose','group_size','pnr_loctr_id', 'pnr_create_dt'])
y = model_data_one_hot['trip_subpurpose']
# dividing X, y into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)


X_resampled, y_resampled = X_train, y_train
#ros = RandomOverSampler(random_state=0)
#X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
#X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)

In [0]:
X.columns.tolist()

In [0]:
# build RF
#build a forest of 200 trees
forest4 = RandomForestClassifier(n_estimators=200,random_state=0, max_depth=17)
forest_model4 = forest4.fit(X_resampled, y_resampled)
RF_feature_name = X.columns
RF_class_name =forest4.classes_


In [0]:
#generate predictions
RF_predictions4 = forest_model4.predict(X_test)
RF_pred_prob4 = forest_model4.predict_proba(X_test)
#calculate performance
RF_predict_prob4 = (np.sum(RF_pred_prob4, axis= 0)/len(y_test))
RF_predict_prob_df4 = pd.DataFrame((RF_predict_prob4), index = RF_class_name)
RF_true_prob_df4 = pd.DataFrame(y_test.value_counts()/len(y_test))
RF_result4 = pd.concat([RF_predict_prob_df4, RF_true_prob_df4], axis=1).reindex(RF_predict_prob_df4.index)
RF_result4.columns = ['Predicted Prob', 'True Prob']
RF_result4['Absolute Bias'] =abs( (RF_result4['Predicted Prob'] - RF_result4['True Prob'] )/RF_result4['True Prob'])
RF_fitting_predictions4 = forest_model4.predict(X_resampled)
print('Label based TRAINING accuracy',accuracy_score(y_resampled, RF_fitting_predictions4) )
print('Label based PREDICTING accuracy is', accuracy_score(y_test, RF_predictions4))
print('Aggregated Probability based PREDICTING accuracy is\n', RF_result4)

In [0]:
precision, recall, fscore, support = precision_recall_fscore_support(y_test, RF_predictions4)
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))

In [0]:
cm7 = confusion_matrix(y_test, RF_predictions4)
disp = ConfusionMatrixDisplay(confusion_matrix=cm7,display_labels=RF_class_name)
disp.plot()

In [0]:
#generate feature importance from RF
feature_importances = pd.DataFrame(forest_model4.feature_importances_, index=X.columns, columns=["Importance"])
feature_importances.sort_values(by='Importance', ascending=False, inplace=True)
feature_importances[:19].plot(kind='bar', figsize=(8,6))

## use TA AA pnr to predict trip intent

In [0]:
ta_aa = fileshare.path(directory_name="ra/projects/RA_Trip_Intent_Model", file_name="ta_aa_pnr_v4_add_mkt.txt")
ta_aa_data = pd.read_csv(ta_aa, sep='	')
ta_aa_data.columns = ta_aa_data.columns.str.lower()
ta_aa_data = ta_aa_data.dropna()
print(ta_aa_data.shape)
print(ta_aa_data.columns)

In [0]:
print(ta_aa_data.columns)

In [0]:
ta_aa_data.pnr_max_cabin_cd.value_counts()

In [0]:
ta_aa_data_train = ta_aa_data[[

'pnr_single_pax_count_indicator',
'ap',
'tkt_min_stay',
'prime_chanl_type_cd',
'tkt_min_stay_category',
'tkt_ap_category',
'pnr_max_cabin_cd',
#'flt_dep_quarter',
#'mkt_group'
]]

In [0]:
ta_aa_data_train.shape

In [0]:
ta_aa_data_one_hot = pd.get_dummies(ta_aa_data_train, columns = [

'prime_chanl_type_cd',
'tkt_min_stay_category',
'tkt_ap_category',
'pnr_max_cabin_cd',
#'flt_dep_quarter',
#'mkt_group'
  ])


In [0]:
ta_aa_data_train.dtypes

In [0]:
ta_aa_data_one_hot.shape

In [0]:
ta_aa_data_one_hot=ta_aa_data_one_hot.dropna()

In [0]:
ta_aa_data_one_hot.shape

In [0]:
RF_predictions_ta_aa = forest_model4.predict(ta_aa_data_one_hot)

In [0]:
RF_predictions_ta_aa.shape

In [0]:
RF_predictions_ta_aa_prob = forest_model4.predict_proba(ta_aa_data_one_hot)

In [0]:
RF_predictions_ta_aa_prob.shape

In [0]:
forest_model4.classes_

In [0]:
#order of column names need to match classes names above
RF_predictions_ta_aa_prob = pd.DataFrame((RF_predictions_ta_aa_prob), columns= ["ra_business_only_prob_pct","ra_vacation_only_prob_pct","ra_vfr_prob_pct"])

In [0]:
RF_predictions_ta_aa_prob.shape

In [0]:
ta_aa_data['ra_predicted_trip_intent'] = RF_predictions_ta_aa

In [0]:
ta_aa_data.shape

In [0]:
ta_aa_data.index = RF_predictions_ta_aa_prob.index

In [0]:
ta_aa_data = pd.concat([ta_aa_data,RF_predictions_ta_aa_prob], axis = 1, ignore_index= False)

In [0]:
ta_aa_data.shape

In [0]:
ta_aa_data

In [0]:
def func_ap_category2(row):
  
  if row['ap'] < 14:
    return '0-13 AP'
  elif row['ap'] < 28:
    return '14-27 AP'
  elif row['ap'] < 50:
    return '28-49 AP'
  elif row['ap'] < 90:
    return '50-89 AP'
  elif row['ap'] >= 90:
    return '90+ AP'
  else:
    return 'Error'
ta_aa_data['tkt_ap_category2'] = ta_aa_data.apply(func_ap_category2, axis = 1)

In [0]:
ta_aa_data

In [0]:
fileshare.to_csv(ta_aa_data, "ra/projects/RA_Trip_Intent_Model/na_ta_aa_data_prediction.txt",sep='\t', index=False)


## use TA AJB pnr to predict trip intent

In [0]:
ta_ajb_pnr = fileshare.path(directory_name="ra/projects/RA_Trip_Intent_Model", file_name="ta_ajb_pnr_v3_add_mkt.txt")
ta_ajb_data = pd.read_csv(ta_ajb_pnr, sep='	')
ta_ajb_data.columns = ta_ajb_data.columns.str.lower()
ta_ajb_data=ta_ajb_data.dropna().copy()
print(ta_ajb_data.shape)
print(ta_ajb_data.columns)

In [0]:
print(ta_ajb_data.columns)

In [0]:
ta_ajb_data.pnr_max_cabin_cd.value_counts()

In [0]:
def pnr_max_cabin_cd(row):
  if row['pnr_max_cabin_cd'] == 'F':
    return 'F'
  elif row['pnr_max_cabin_cd'] == 'C':
    return 'C'
  elif row['pnr_max_cabin_cd'] == 'W':
    return 'W'
  elif row['pnr_max_cabin_cd'] == 'Y':
    return 'Y'
  else:
    return 'Y'
ta_ajb_data['pnr_max_cabin_cd'] = ta_ajb_data.apply(pnr_max_cabin_cd, axis = 1)

In [0]:
ta_ajb_data_train = ta_ajb_data[[
  
'pnr_single_pax_count_indicator',
'ap',
'tkt_min_stay',
'prime_chanl_type_cd',
'tkt_min_stay_category',
'tkt_ap_category',
'pnr_max_cabin_cd'
]]

In [0]:
ta_ajb_data_train.shape

In [0]:
ta_ajb_data_one_hot = pd.get_dummies(ta_ajb_data_train, columns = [

'prime_chanl_type_cd',
'tkt_min_stay_category',
'tkt_ap_category',
'pnr_max_cabin_cd'

  ])
#ta_ajb_data_one_hot['prime_chanl_type_cd_ INC\""'] = 0
#ta_ajb_data_one_hot['tkt_min_stay_category_Same Day Roundtrip'] = 0

In [0]:
ta_ajb_data_one_hot.shape

In [0]:
ta_ajb_data.shape

In [0]:
ta_ajb_data_one_hot=ta_ajb_data_one_hot.dropna().copy()

In [0]:
RF_predictions_ta_ajb = forest_model4.predict(ta_ajb_data_one_hot)

In [0]:
RF_predictions_ta_ajb.shape

In [0]:
RF_predictions_ta_ajb_prob = forest_model4.predict_proba(ta_ajb_data_one_hot)

In [0]:
RF_predictions_ta_ajb_prob.shape

In [0]:
RF_predictions_ta_ajb_prob = pd.DataFrame((RF_predictions_ta_ajb_prob), columns= RF_class_name)

In [0]:
RF_predictions_ta_ajb_prob.shape

In [0]:
#check if ajb data and one hot data order match
#ta_ajb_data = pd.concat([ta_ajb_data,ta_ajb_data_one_hot], axis = 1)

In [0]:
ta_ajb_data['predicted_trip_intent'] = RF_predictions_ta_ajb

In [0]:
ta_ajb_data.shape

In [0]:
ta_ajb_data.index = RF_predictions_ta_ajb_prob.index

In [0]:
ta_ajb_data = pd.concat([ta_ajb_data,RF_predictions_ta_ajb_prob], axis = 1)

In [0]:
ta_ajb_data.shape

In [0]:
def func_ap_category2(row):
  
  if row['ap'] < 14:
    return '0-13 AP'
  elif row['ap'] < 28:
    return '14-27 AP'
  elif row['ap'] < 50:
    return '28-49 AP'
  elif row['ap'] < 90:
    return '50-89 AP'
  elif row['ap'] >= 90:
    return '90+ AP'
  else:
    return 'Error'
ta_ajb_data['tkt_ap_category2'] = ta_ajb_data.apply(func_ap_category2, axis = 1)

In [0]:
ta_ajb_data

In [0]:
ta_ajb_data=ta_ajb_data.dropna()
ta_ajb_data.shape

In [0]:
ta_ajb_data.mkt.value_counts()

In [0]:
ta_ajb_prediction_data=spark.createDataFrame(ta_ajb_data)

In [0]:
ta_ajb_prediction_data.createOrReplaceTempView("ta_ajb_prediction_data")

In [0]:
od = mos.read_sql("""
 SELECT

PNR_LOCTR_ID,
cast(PNR_CREATE_DT as date format 'mm/dd/YYYY')(VARCHAR(10)) AS PNR_CREATE_DT,

CASE
    WHEN (Trim(OD_DEP_CNTRY_CD) IN ('AU') OR Trim(OD_ARVL_CNTRY_CD) IN ('AU')) THEN 'Australia'
    WHEN (Trim(OD_DEP_CNTRY_CD) IN ('NZ') OR Trim(OD_ARVL_CNTRY_CD) IN ('NZ')) THEN 'New Zealand'
    WHEN (Trim(OD_DEP_CNTRY_CD) IN ('CN') OR Trim(OD_ARVL_CNTRY_CD) IN ('CN')) THEN 'China'
    WHEN (Trim(OD_DEP_CNTRY_CD) IN ('KR') OR Trim(OD_ARVL_CNTRY_CD) IN ('KR')) THEN 'Korea'
    WHEN (Trim(OD_DEP_CNTRY_CD) IN ('JP') OR Trim(OD_ARVL_CNTRY_CD) IN ('JP')) THEN 'Japan'
    WHEN (Trim(OD_DEP_CNTRY_CD) IN ('IN') OR Trim(OD_ARVL_CNTRY_CD) IN ('IN')) THEN 'India'
    WHEN ((Trim(OD_DEP_CNTRY_CD) IN ('US','MX') AND Trim(OD_ARVL_CNTRY_CD) IN ('AO','DJ','ET','KE','MZ','NA','RW','SC','SO','SD','TZ','UG','ZM','ZW'))
        OR (Trim(OD_DEP_CNTRY_CD) IN ('AO','DJ','ET','KE','MZ','NA','RW','SC','SO','SD','TZ','UG','ZM','ZW') AND Trim(OD_ARVL_CNTRY_CD) IN ('US','MX'))) THEN 'Africa'
    WHEN ((Trim(OD_DEP_CNTRY_CD) IN ('US','MX') AND Trim(OD_ARVL_CNTRY_CD) IN ('IQ','JO','KW','LB','OM','QA','SA','AE','AM','AZ','GE','UZ','KZ'))
        OR (Trim(OD_DEP_CNTRY_CD) IN ('IQ','JO','KW','LB','OM','QA','SA','AE','AM','AZ','GE','UZ','KZ') AND Trim(OD_ARVL_CNTRY_CD) IN ('US','MX'))) THEN 'Middle East'
    WHEN ((Trim(OD_DEP_CNTRY_CD) IN ('US','MX') AND Trim(OD_ARVL_CNTRY_CD) IN ('AF','BD','IN','MV','NP','PK','LK'))
        OR (Trim(OD_DEP_CNTRY_CD) IN ('AF','BD','IN','MV','NP','PK','LK') AND Trim(OD_ARVL_CNTRY_CD) IN ('US','MX'))) THEN 'Subcontinent'
    WHEN (Trim(OD_DEP_CNTRY_CD) IN ('GB','UK') OR Trim(OD_ARVL_CNTRY_CD) IN ('GB','UK')) THEN 'UK'
    WHEN (Trim(OD_DEP_CNTRY_CD) IN ('ES') OR Trim(OD_ARVL_CNTRY_CD) IN ('ES')) THEN 'Spain'
    WHEN (Trim(OD_DEP_CNTRY_CD) IN ('IL') OR Trim(OD_ARVL_CNTRY_CD) IN ('IL')) THEN 'Israel'
    WHEN (Trim(OD_DEP_CNTRY_CD) IN ('AT','BE','BG','CH','CY','CZ','DE','DK','EE','FI','FR','GG','GI','GR','HR','HU','IE','IM','IS','IT','JE','LT','LU','LV','MT','NL','NO','PL','PT','RO','SE','SI','SK') 
        OR Trim(OD_ARVL_CNTRY_CD) IN ('AT','BE','BG','CH','CY','CZ','DE','DK','EE','FI','FR','GG','GI','GR','HR','HU','IE','IM','IS','IT','JE','LT','LU','LV','MT','NL','NO','PL','PT','RO','SE','SI','SK')) THEN 'Core Europe'
    ELSE 'OTHER' 
END AS INTL_REGION,
CASE
WHEN OD_DEP_CNTRY_CD IN ('US','CA','PR','VI') THEN Trim(OD_ORIGIN_CITY_METRO_CD)||Trim(OD_DESTNTN_CITY_METRO_CD)
ELSE Trim(OD_DESTNTN_CITY_METRO_CD)||Trim(OD_ORIGIN_CITY_METRO_CD) END AS MKT
FROM PROD_JBA_VW.JBA_BKG_OD_DETAIL
WHERE 1=1
AND OD_DEP_DT BETWEEN '2022-10-01' AND '2023-09-30'
AND PNR_CREATE_DT >= '2021-10-01'
AND BKG_POS_CNTRY_CD IN ('US','CA','PR','VI')
AND INTL_REGION IN ('UK','Spain','Core Europe','Israel')
AND SNPSHT_DT >=   '2021-10-01'                
QUALIFY Row_Number () Over (PARTITION BY PNR_LOCTR_ID, PNR_CREATE_DT ORDER BY OD_TTL_NET_YQ_USD_AMT DESC) = 1          
 """)
od.createOrReplaceTempView("od")


In [0]:
%sql
select MKT, COUNT(*) AS PNR_CNT from od GROUP BY 1 ORDER BY 2 DESC

In [0]:
%sql select * from od

In [0]:
%sql select * from ta_ajb_prediction_data

In [0]:
output = spark.sql(
  """
  select
  a.*,
  case
  WHEN trim(b.MKT) IN (

'DFWLON','MSYLON','CLTLON','PDXLON','RDULON','CVGLON','PHLLON','BWILON','CHIVCE','PHLVCE','SFOMAD','LAXMAD','CHIMAD',
'CLTMAD','PHLATH','PHLNAP','PHLLIS','PHLDUB','CHIATH','CHIHEL','BOSMAD','PHLMAD','CHILON','BOSLON','MIALON','LAXLON',
'CHIBCN','NYCBCN','DFWPAR','CHIPAR','CHIROM','NYCROM'
) THEN trim(b.MKT)
ELSE 'OTHER'
END AS MODIFIED_MKT
  FROM ta_ajb_prediction_data a
  left join od b
  on TRIM(a.pnr_loctr_id) = TRIM(b.PNR_LOCTR_ID) and a.pnr_create_dt  = b.PNR_CREATE_DT
"""
)
output.createOrReplaceTempView("output")

In [0]:
%sql
select MODIFIED_MKT, COUNT(*) AS PNR_CNT from output GROUP BY 1 ORDER BY 2 DESC

In [0]:
%sql

DROP TABLE IF EXISTS rm_workspace.ta_ajb_prediction_with_mkt;
CREATE TABLE rm_workspace.ta_ajb_prediction_data
USING DELTA
LOCATION "abfss://rmsbx@aabaoriondlsprod.dfs.core.windows.net/struct/chengao/ta_ajb_prediction_with_mkt"

AS (

  SELECT * FROM output
)

In [0]:
fileshare.to_csv(output, "ra/projects/RA_Trip_Intent_Model/na_ta_ajb_data_prediction_with_mkt.txt",sep='\t', index=False)
